# Installation

In [ ]:
%%bash
pip install  --upgrade\
    'vllm>=0.8.2' \
    'transformers>=4.50.3' \
    pyzmq \
    unsloth \
    accelerate \
    bitsandbytes \
    openai \
    langchain-text-splitters \
    peft \
    FlagEmbedding \
    datasets \
    faiss-cpu \
    langchain-text-splitters \
    tavily-python \
    "flashinfer-python>=0.2.4"  --extra-index-url https://flashinfer.ai/whl/cu124/torch2.6/
git clone https://github.com/ggml-org/llama.cpp.git
cd llama.cpp/gguf-py/ && pip install --editable .
pip install jupyter-kernel-gateway ipykernel
pip install --upgrade --no-deps numpy==1.26.4 pandas==2.2.2

In [ ]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
!huggingface-cli login --add-to-git-credential --token $HF_TOKEN
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

```bash
VLLM_BACKEND=FLASHINFER VLLM_USE_V1=1 VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 TOKENIZERS_PARALLELISM=true MAX_JOBS=2 vllm serve ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g --port 8877 --max-model-len 4096 --api-key token-abc123 --quantization compressed-tensors --max-num-seqs=1
```

```bash
VLLM_BACKEND=FLASHINFER VLLM_USE_V1=1 VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 TOKENIZERS_PARALLELISM=true MAX_JOBS=2 vllm serve microsoft/bitnet-b1.58-2B-4T --port 8877 --max-model-len 4096 --api-key token-abc123 --quantization compressed-tensors --max-num-seqs=1
```

# Web Search Agent

In [4]:
from tavily import TavilyClient
import asyncio, os, requests, time, json
from IPython.display import display, Markdown, Latex

tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [5]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [6]:
def deduplicate_and_format_sources(search_response, max_tokens_per_source, include_raw_content=True):
     # Collect all results
    sources_list = []
    for response in search_response:
        sources_list.extend(response['results'])

    # Deduplicate by URL
    unique_sources = {source['url']: source for source in sources_list}

    # Format output
    formatted_text = "Content from sources:\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"{'='*80}\n"  # Clear section separator
        formatted_text += f"Source: {source['title']}\n"
        formatted_text += f"{'-'*80}\n"  # Subsection separator
        formatted_text += f"URL: {source['url']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['content']}\n===\n"
        if include_raw_content:
            # Using rough estimate of 4 characters per token
            char_limit = max_tokens_per_source * 4
            # Handle None raw_content
            raw_content = source.get('raw_content', '')
            if raw_content is None:
                raw_content = ''
                print(f"Warning: No raw_content found for source {source['url']}")
            if len(raw_content) > char_limit:
                raw_content = raw_content[:char_limit] + "... [truncated]"
            formatted_text += f"Full source content limited to {max_tokens_per_source} tokens: {raw_content}\n\n"
        formatted_text += f"{'='*80}\n\n" # End section separator

    return formatted_text.strip()

In [7]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "Llama-3.2-3B-Instruct",
        messages = message_list,
        max_tokens=2048,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [8]:
import threading

def worker(query, search_result, req_num_result, include_raw, req_topic):
    print(f"Thread: {query}")
    search_result.append(
        tavily_client.search(
            query,
            max_results= req_num_result,
            include_raw_content= include_raw,
            topic= req_topic
        )
    )

In [9]:
def ask_tavily(search_queries, search_tasks, req_num_result, include_raw, req_topic):
    start_time = time.time()
    threads = []

    for query in search_queries:
        t = threading.Thread(target=worker, args=(query, search_tasks, req_num_result, include_raw, req_topic))
        threads.append(t)
        t.start()

    for thread in threads:
        thread.join()

    end_time = time.time()
    execution_time = end_time - start_time

    print(f"\nask_tavily task running time: {execution_time:.2f}초 \n")

In [10]:
def ask_plan_query_writer(topic, content):
    llm_prompt = """You are an expert technical writer crafting a section that synthesizes information
<section topic>
""" + topic + """
</section topic>

<section organization>
""" + content + """
</section organization>

<Task>
Your goal is to generate 3 web search queries that will help gather information for planning the sections.

The queries should:

1. Be related to the section topic
2. Help satisfy the requirements specified in the section organization

Make the queries specific enough to find high-quality, relevant sources while covering the breadth needed for the section structure.

Note1. that today's date is """+time.strftime("%Y-%m-%d")+""".
Note2. Output your response in JSON format, with the following structure: { "queries": [ "query1", "query2", "query3" ] }
Only output in JSON format when generating responses. Never include additional phrases such as "here is content in JSON format".
</Task>"""

    return llm_prompt

In [ ]:
def ask_final_writer_instructions(topic, content, search_tasks):
    final_section_writer="""You are an expert technical writer.

<Section name>
""" + content + """
</Section name>

<Section topic>
""" + topic + """
</Section topic>

<Available Website Search Content>
""" + deduplicate_and_format_sources(search_tasks, max_tokens_per_source=4000, include_raw_content=True) + """
</Available Website Search Content>

<Task>
1. Section-Specific Approach:

For Introduction:
- Use # for Website Search title (Markdown format)
- Write in simple and clear language
- Focus on the core motivation for the Section in 1-2 paragraphs
- Use a clear narrative arc to introduce the Section
- Include NO structural elements (no lists or tables)
- No sources section needed

For Conclusion/Summary:
- Use ## for Conclusion/Summary title (Markdown format)
- For comparative Conclusion/Summary:
    * Must include a focused comparison table using Markdown table syntax
    * Table should distill insights from the Section
    * Keep table entries clear and concise
- For non-comparative Conclusion/Summary:
    * Only use ONE structural element IF it helps distill the points made in the Section:
    * Either a focused table comparing items present in the Section (using Markdown table syntax)
    * Or a short list using proper Markdown list syntax:
      - Use `*` or `-` for unordered lists
      - Use `1.` for ordered lists
      - Ensure proper indentation and spacing
- Sources and url section needed. (especially when expressing a URL, please provide the entire URL exactly as given in the content without abbreviating it.)
- End with specific next steps or implications

2. Writing Approach:
- Use concrete details over general statements
- Make every word count
- Focus on your single most important point
</Task>

<Quality Checks>
- Verify that EVERY claim is grounded in the provided Source material
- Confirm each URL appears ONLY ONCE in the Source list
- For introduction: # for Website Search title, no structural elements, no sources section
- For conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section
- Markdown format
- Do not include word count or any preamble in your response
</Quality Checks>

Please note that respond in Korean always."""

    return final_section_writer

In [ ]:
system_prompt = "You are a helpful assistant. "

topic = "판례확인"
content = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ask_plan_query_writer(topic, content)},
    ]

response_query = generate_response(messages)

{"queries": ["부하직원 비위사실 판결", "지방공무원법 부하직원 비위사실", "부하직원 비위사실 위법性 판결"]}

inference time: 1.41282 sec 




In [ ]:
json_data = json.loads(response_query)
queries = json_data['queries']

print("사용자 발화 기반으로 추출한 web query 문장 3건:")
print(queries)

search_tasks = []
req_topic = 'general' # news   gerneral 과 news 중 선택
req_num_result = 3    # 각 web query 에 대해 리턴할 site 개수
include_raw = False    # site 의 원본 컨텐츠 리턴 유무

ask_tavily(queries, search_tasks, req_num_result, include_raw, req_topic)
print(search_tasks)

사용자 발화 기반으로 추출한 web query 문장 3건:
['부하직원 비위사실 판결', '지방공무원법 부하직원 비위사실', '부하직원 비위사실 위법性 판결']
Thread: 부하직원 비위사실 판결
Thread: 지방공무원법 부하직원 비위사실
Thread: 부하직원 비위사실 위법性 판결

ask_tavily task running time: 4.39초 

[{'query': '부하직원 비위사실 위법性 판결', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그', 'url': 'https://m.blog.naver.com/pdsph1004/222061319284', 'content': '22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실', 'score': 0.65146625, 'raw_content': None}, {'title': '대법원 2009다2545 - CaseNote - 케이스노트', 'url': 'https://casenote.kr/대법원/2009다2545', 'content': '선고 2009다2545 판결 [손해배상(기)] ... 대표이사가 해당 사업부서 등의 임원들에게 적절한 조치를 취하도록 지시하는 이상으로 부하직원들의 위법행위를 방지하기 위한 어떠한 구체적인 직무를 수행할 의무를 가진다고 보기 어렵다고 한 사례 ... 상급자들의 부당', 'score': 0.46337196, 'raw_content': None}, {'title': '정당한 징계사유가 존재하는지 여부 등이 문제된 사건[대법원 2021. 11. 25. 선고 중요판결] - 판례속보', 'url': 'https:/

In [ ]:
search_tasks

[{'query': '부하직원 비위사실 위법性 판결',
  'follow_up_questions': None,
  'answer': None,
  'images': [],
  'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그',
    'url': 'https://m.blog.naver.com/pdsph1004/222061319284',
    'content': '22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실',
    'score': 0.65146625,
    'raw_content': None},
   {'title': '대법원 2009다2545 - CaseNote - 케이스노트',
    'url': 'https://casenote.kr/대법원/2009다2545',
    'content': '선고 2009다2545 판결 [손해배상(기)] ... 대표이사가 해당 사업부서 등의 임원들에게 적절한 조치를 취하도록 지시하는 이상으로 부하직원들의 위법행위를 방지하기 위한 어떠한 구체적인 직무를 수행할 의무를 가진다고 보기 어렵다고 한 사례 ... 상급자들의 부당',
    'score': 0.46337196,
    'raw_content': None},
   {'title': '정당한 징계사유가 존재하는지 여부 등이 문제된 사건[대법원 2021. 11. 25. 선고 중요판결] - 판례속보',
    'url': 'https://www.scourt.go.kr/portal/news/NewsViewAction.work?seqnum=8099&gubun=4',
    'content': '6. 25. 선고 2016두56042 판결 등 참조). 취업규칙은 노사 간의 집단적인 법률관계를 규정하는 법규범

In [ ]:
print("\n\n=================================================================\n")
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
4) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  content + "\n후보문장: " + str(search_tasks)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response_query = generate_response(messages)

print("\n\n=========================  Search Report  ========================================\n")
display(Markdown(response_query))




질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?

해당 질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"는 대법원 판결에 의하면, 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로, 아무리 부하직원에게 파면에 해당하는 비위사실이 있다 하더라도, 감독자는 위법을 인정하지 않는다.
  - 대법원 2009다2545 판결 등 참조. 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?는 대法원 판결에 의하면,부하직원이 징계에 해당하는 사안에 대하여는 수사기관의 비위 사실을 증명해야 하며, 그로 인해 상급자들의 부당한 징계가 발생할 수 있으므로, 정당한 징계사유가 존재하는지 여부가 문제된 사건이다.
  - 대法원 2021.11.25 선고 중요판결 등 참조.

해당 질문에 대한 답변 후보 문장들 중에서 가장 적합한 후보는: 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위 pháp을 인정하지 않는다."

inference time: 14.62169 sec 




=========================  Search Report  ========================================



질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?

해당 질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"는 대법원 판결에 의하면, 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로, 아무리 부하직원에게 파면에 해당하는 비위사실이 있다 하더라도, 감독자는 위법을 인정하지 않는다.
  - 대법원 2009다2545 판결 등 참조. 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?는 대法원 판결에 의하면,부하직원이 징계에 해당하는 사안에 대하여는 수사기관의 비위 사실을 증명해야 하며, 그로 인해 상급자들의 부당한 징계가 발생할 수 있으므로, 정당한 징계사유가 존재하는지 여부가 문제된 사건이다.
  - 대法원 2021.11.25 선고 중요판결 등 참조.

해당 질문에 대한 답변 후보 문장들 중에서 가장 적합한 후보는: 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위 pháp을 인정하지 않는다."

# Project

# [Mini Project]

## ㅇㅇㅇ 지능을 갖고있는 Agent 만들기

  - 오늘은 그간 실습했던 prompt engineering, Tool 활용, RAG, PEFT 등을 활용하여 각자 관심있는 영역에대해 질의응답할 수 있는 Agent를 만들어보겠습니다.
  - llama3.1 에게 LLM 기반 agent 개발할때 어떤 step 이 필요한지 확인해봐주세요.

---

In [ ]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

# model="Llama-3.2-3B-Instruct",
# model="gemma-3",

def vllm_generate(system_prompt, user_prompt):
    completion = client.chat.completions.create(
        model="Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.95,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [ ]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = "LLM을 활용한 Agent 를 개발할때 어떤 step 이 필요한지 개조식으로 설명해줘."

vllm_generate(system_prompt, user_prompt)

## LLM 기반 에이전트 개발 단계 (개조식)

LLM(Large Language Model)을 활용한 에이전트 개발은 복잡하지만, 체계적인 단계를 거치면 성공적인 결과물을 얻을 수 있습니다. 다음은 주요 단계들을 개조식으로 정리한 것입니다.

**1. 기획 및 정의 (Planning & Definition)**

*   **1.1. 에이전트 목표 정의:**
    *   에이전트가 어떤 문제를 해결하거나 어떤 작업을 수행해야 하는지 명확하게 정의합니다. (예: 고객 문의 응대, 이메일 작성, 데이터 분석 보고서 생성 등)
*   **1.2. 에이전트 역할 및 책임 설정:**
    *   에이전트가 어떤 역할을 수행하고, 어떤 책임을 지는지 구체적으로 설정합니다. (예: 정보 검색자, 의사 결정자, 실행자 등)
*   **1.3. 사용 LLM 선정:**
    *   목표와 기능에 적합한 LLM을 선택합니다. (예: GPT-4, Gemini, Claude 등) 비용, 성능, API 접근성 등을 고려합니다.
*   **1.4. 핵심 기능 및 워크플로우 설계:**
    *   에이전트가 수행할 핵심 기능들을 정의하고, 이를 연결하는 워크플로우를 설계합니다. (예: 사용자 입력 -> LLM 질문 생성 -> 정보 검색 -> 답변 생성 -> 사용자 출력)

**2. 개발 환경 구축 (Environment Setup)**

*   **2.1. 개발 환경 설정:**
    *   Python, LLM API 키, 필요한 라이브러리 (LangChain, LlamaIndex 등) 설치 및 설정합니다.
*   **2.2. 데이터 준비:**
    *   에이전트가 필요로 하는 데이터 (문서, 데이터베이스, API 등)를 수집하고, LLM이 이해할 수 있는 형태로 가공합니다.
*   **2.3. 프롬프트 엔지니어링 준비:**
    *   LLM에게 효과적인 지시를 내리기 위한 프롬프트 템플릿을 설계합니다.

**3. 핵심 컴포넌트 개발 (Core Component Deve

---
## 문제 정의 & 데이터 수집 및 전처리

  - Agent 개발 첫 step : 뭐니뭐니 해도 Data  
    [AI Hub] https://aihub.or.kr/
    huggingface 의 dataset

    국내데이터는 AI Hub 에도 전처리 잘돼 있는 데이터셋 들이 많이 있습니다.  
    ex) 법률 - 상황에 따른 판례 데이터
    https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71723  

    각 판례가 json 형태로 정리돼 있는 데이터 입니다.

---

In [ ]:
user_prompt = """교통계 순경의 파면사례에 대해 알려줘. 답변할땐 참고문장을 반드시 참고해서 답변해줘. \n
참고문장
{
  "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93872&type=HTML&mobileYn=",
  "판시사항": "【판시사항】\n\n교통계순경이 주차금지 구역의 주차를 묵인하고 그 위반자로부터 이조회화액자 일점을 교부받은 비위사실을 징계사유로 한 파면처분이 징계재량권의 범위를 넘어선 것인지 여부",
  "판결요지": "【판결요지】\n\n교통계 순경 (갑)이 주차금지구역의 주차를 단속하지 않은 장소가 교통에 크게 지장을 주는 장소도 아니고 주차위반자인 (을)로부터 교부받았다는 이조회화액자 1점은 (을)의 소속회사에서 고객을 위하여 만든 특별 증정품에 불과하며 그리고 (갑)은 여러차례 표창을 받은 사실이 있는 점 등을 종합하면 위 비위사실만 가지고 (갑)에 대하여 징계처분 중 가장 무거운 파면처분을 택한 것은 징계재량권의 범위를 일탈하였다고 할 것이다.",
  "참조조문": "【참조조문】\n\n경찰공무원법 제53조 제1항 제1호\n\n제53조 제1항 제3호",
  "참조판례": null,
  "판례내용": "【전문】\n\n【원고, 상고인】 원고 소송대리인 변호사 이해우\n\n【피고, 피상고인】 내무부장관\n\n【원 판 결】 서울고등법원 1976.9.21. 선고 76구181 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원에 환송한다.\n\n【이유】\n\n원고 소송대리인의 상고이유(상고이유보충서 기재 사실은 법정기간내에 제출된 상고이유를 부언하는 한도내에서 참작한다)를 판단한다.\n\n원판결의 설시이유에 의하면, 원심은 원고가 ○○경찰서 경비과 교통계 순경으로 ○○△가 □□□ 앞 로타리에서 교통정리 및 위법차량 등의 단속업무에 종사하고 있었던 바, 전부터 안면이 있는 소외인이 원고의 업무구역내인 □□□ 옆 주차금지구역에 자주 주차하여도 원고는 위 소외인과의 안면을 생각하여 교통법규위반자로 적발하지 않았고 이에 위 소외인은 평소 원고에 대하여 미안한 생각이 있어 1975.10.14 15:55경 역시 위 □□□ 옆 골목에 위법주차 중 마침 그 곳을 지나가던 원고에게 동 소외인 근무회사가 고객을 위하여 특별증정품으로 만들어 둔 이조회화 액자 1점을 교부한 사실을 인정한 후, 원고의 위와 같은 비위는 경찰공무원법 제53조 제1항 제1호및 제3호소정의 징계사유에 해당하는 것이며 이건 파면처분이 징계재량권의 범위를 넘어선 것이라고는 볼 수 없다고 판시하고 있다.\n\n살피건대 원판결이 들고 있는 증거들을 기록에 비추어 검토하여 보면 원판결이 그 판시와 같이 징계사유에 해당하는 비위사실이 있음을 인정한 과정에는 아무런 위법사유가 있다고 할 수 없다. 그러니 원판결에 심리미진으로 인한 사실오인이 있다는 상고논지 제1점은 이유없다.\n\n그러나원고가 여러차례 표창을 받은 사실과 원고가 받았다는 위 액자가 위 소외인 소속회사에서 고객을 위하여 만든 특별 증정품에 불과한 것임은 원심도 인정하고 있는 바이고 또 본건 단속대상 차량이 위법 주차한 장소가 교통에 크게 지장을 주는 장소임을 긍인하게 할 자료를 기록상 찾아 볼 수 없는 본건에 있어서, 원고에 대하여 징계처분 중 가장 무거운 파면처분을 택한 것은 징계재량권의 범위를 일탈한 것이라 할 것이다.그렇다면 원판결은 심리미진으로 인한 이유불비 내지는 징계재량권의 범위에 관한 법리를 오해한 위법을 범한 것이라고 할 것이니 이 점에서 원판결은 파기를 면하지 못할 것이고 이점을 지적하는 상고 논지 제2점은 이유있다.\n\n그러므로 원판결을 파기하고 사건을 원심법원에 환송하기로 하여 관여법관의 일치된 의견으로 주문과 같이 판결한다.\n\n대법관   한환진(재판장) 민문기 안병수 강안희"
}"""

vllm_generate(system_prompt, user_prompt)

네, 교통계 순경의 파면 사례에 대해 자세히 알려드리겠습니다. 위에 제공해주신 판례를 바탕으로, 파면 사유가 되는 주요 사례와 그 배경을 설명드리겠습니다.

**교통계 순경 파면 사례 주요 내용**

제공된 판례는 1976년 사건으로, 당시의 법리적 판단 기준을 보여주지만, 현재는 법률 및 사회적 인식 변화로 인해 적용이 달라졌을 수 있습니다. 하지만 핵심적인 내용을 중심으로 설명드리겠습니다.

*   **주요 비위 사실:**
    *   주차 금지 구역의 주차를 묵인하는 행위
    *   주차 위반자로부터 이조회화액자를 교부받는 행위 (특별 증정품)
*   **파면 사유 판단:**
    *   원심 법원은 이와 같은 비위 사실만으로는 파면 처분이 징계 재량권의 범위를 넘어선다고 보지 않았습니다.
    *   그러나 대법원은 원고가 여러 차례 표창을 받은 점, 이조회화액자가 특별 증정품이라는 점, 해당 주차 구역이 교통에 크게 지장을 주는 장소는 아니라는 점 등을 종합적으로 고려하여 징계 재량권의 범위를 일탈했다고 판단했습니다.
    *   따라서 원판결을 파기하고 사건을 환송하기로 결정했습니다.

**현재의 파면 사유 (참고)**

현재 경찰공무원법 제53조 제1항 제1호 및 제3호에 명시된 징계 사유를 통해 파면 사유를 판단합니다. 주요 파면 사유는 다음과 같습니다.

1.  **경찰공무원법 제53조 제1항 제1호: 직무 집행상의 중대한 과실:** 업무 수행 과정에서 법령이나 규정을 위반하거나, 업무상 중요한 의무를 소홀히 하여 국민의 안전과 재산에 심각한 피해를 야기한 경우입니다. 예를 들어, 교통 단속 중 사고 발생 시 적절한 조치를 취하지 않아 사고 피해가 커진 경우 등이 해당될 수 있습니다.
2.  **경찰공무원법 제53조 제1항 제3호: 부정부패:** 뇌물수수, 강요, 금품갈취 등 공정성을 훼손하는 행위입니다.
3.  **기타:** 위에서 언급된 사유 외에도, 경찰 조직의 신뢰를 떨어뜨리는 중대한 비위 행위는 파면 사유가 될 수 있습니다.

In [ ]:
user_prompt="linux 디렉토리에 있는 여러개의 json 파일을 하나씩 pyhon 으로 읽어들여 text list 형태로 저장하는 코드 알려줘."
vllm_generate(system_prompt, user_prompt)

네, Linux 디렉토리에 있는 여러 개의 JSON 파일을 하나씩 Python으로 읽어들여 텍스트 리스트 형태로 저장하는 코드입니다.

```python
import os
import json

def json_files_to_text_list(directory):
    """
    지정된 디렉토리의 모든 JSON 파일을 읽어들여 텍스트 리스트 형태로 반환합니다.

    Args:
        directory (str): JSON 파일이 있는 디렉토리 경로.

    Returns:
        list: 각 JSON 파일의 내용을 문자열로 담은 리스트.
    """

    text_list = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    text_list.append(json.dumps(data, ensure_ascii=False, indent=4))  # JSON을 문자열로 변환
            except Exception as e:
                print(f"파일 읽기 오류: {filename} - {e}")
    return text_list

# 사용 예시
directory_path = "/path/to/your/directory"  # 실제 디렉토리 경로로 변경하세요.
result = json_files_to_text_list(directory_path)

for i, text in enumerate(result):
    print(f"파일 {i+1}:"

In [ ]:
import json
import os

# 디렉토리 경로를 입력하세요
directory_path = 'data_test/'
text_list = []

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            text_list.append(text_json)
    else:
        continue


In [ ]:
text_list[0]

'{\n    "판례일련번호": 104645,\n    "사건명": "근로기준법위반",\n    "사건번호": "88도1162",\n    "선고일자": "1988-11-22",\n    "법원명": "대법원",\n    "법원종류코드": null,\n    "사건종류명": "형사",\n    "사건종류코드": 400102,\n    "판결유형": "판결",\n    "선고": "선고",\n    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=104645&type=HTML&mobileYn=",\n    "판시사항": "【판시사항】\\n근로기준법 제15조의 사업경영담당자의 의미",\n    "판결요지": "【판결요지】\\n근로기준법 제15조소정의 근로자에게 임금지급의무를 부담하는 사업경영담당자라 함은 사업경영일반에 관하여 책임을 지는 자로서 사업주로부터 사업경영의 전부 또는 일부에 대하여 포괄적인 위임을 받고 대외적으로 사업을 대표하거나 대리하는 자를 말한다.",\n    "참조조문": "【참조조문】\\n근로기준법 제15조",\n    "참조판례": null,\n    "판례내용": "【피고인】 피고인 1 외 2인\\n【상고인】 각 검사\\n【원심판결】 서울형사지방법원 1988.3.2. 선고 87노6626 판결\\n【주문】\\n상고를 기각한다.\\n【이유】\\n상고이유를 본다.\\n원심이 유지한 제1심판결에 의하면근로기준법상 근로자에게 임금지급의무를 부담하는 사업경영담당자( 근로기준법 제15조)라 함은 사업경영 일반에 관하여 책임을 지는 자로서 사업주로부터 사업경영의 전부 또는 일부에 대하여 포괄적인 위임을 받고 대외적으로 사업을 대표하거나 대리하는 자를 말한다고 판시하고나아가 그 거시의 증거들을 종합하여 판시 기간동안에는 피고인 1은 공소외 주식회사의 생산관리이사로서 책장 및 걸상 등을 제조하는 공장의 업무관리를, 피고 2는 영업담당이사로서 영업관계 업무를, 피고 3은 감사 겸 기획실장으로서 기획업무를 담당

In [ ]:
directory_path = 'data_test/'
data_list = []
count = 0

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            data_list.append(text_json)
            if count % 10 ==0:
                print(". ", end="")
            count += 1
    else:
        continue

. . . . . . . . . . . . . . . . 

In [ ]:
len(data_list)

151

---
# RAG 준비  
## BGE-M3 모델로 준비된 데이터셋을 embedding 합니다.  
  - 현재 GPU instance 에서 nvidia-smi 를 수행해봐주세요 (gpu mem 확인)
  - vllm 으로 모델 서빙하고 있던것이 있다면 잠시 vllm 은 내려주세요.
      
mem 확보 후 BGE-Me 모델을 gpu 에 로드합니다.  

---

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch, os, faiss
from FlagEmbedding import FlagModel
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
raw_model = AutoModel.from_pretrained("BAAI/bge-m3")

/home/freenak/miniconda3/envs/llm_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# get the BGE embedding model
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

In [ ]:
# get the embedding of the corpus
law_embeddings = model.encode(data_list)

print("shape of the corpus embeddings:", law_embeddings.shape)
print("data type of the embeddings: ", law_embeddings.dtype)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


shape of the corpus embeddings: (151, 1024)
data type of the embeddings:  float16


---
## embedding 완료 후 faiss index 로 embedding 등록  
  - law_embedding 을 바로 사용하면 안되는건가?

---

In [ ]:
law_embeddings_np = law_embeddings.astype(np.float32)
dim = law_embeddings_np.shape[-1]

# create the faiss index and store the corpus embeddings into the vector space
index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)

In [ ]:
print(index.is_trained)

# add all the vectors to the index
index.add(law_embeddings_np)

print(f"total number of vectors: {index.ntotal}")

True
total number of vectors: 151


In [ ]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

query_embeddings = model.encode_queries(queries)
dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    print(f"RAG_{count}:\t{data_list[ids[0][i]]}\n")
    count = count+1

In [ ]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

rag_docs = ''
query_embeddings = model.encode_queries(queries)
dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

print(rag_docs)

In [ ]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 그리고 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
3) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

user_prompt = question + "\
\n후보문장: \
" + rag_docs

In [ ]:
user_prompt

'지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?\n후보문장: 후보문장_0:\t{\n    "판례일련번호": 93952,\n    "사건명": "감봉처분취소",\n    "사건번호": "78누164",\n    "선고일자": "1978-08-22",\n    "법원명": "대법원",\n    "법원종류코드": null,\n    "사건종류명": "일반행정",\n    "사건종류코드": 400107.0,\n    "판결유형": "판결",\n    "선고": "선고",\n    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",\n    "판시사항": "【판시사항】\\n\\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",\n    "판결요지": "【판결요지】\\n\\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",\n    "참조조문": "【참조조문】\\n\\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",\n    "참조판례": null,\n    "판례내용": "【전문】\\n\\n【원고, 상고인】 원고\\n\\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\\n\\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\\n\\n【주문】\\n\\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\\n\\n【이유】\\n\\n원고의 상고이유를 판단한다.\\n\\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕

---
## vllm 으로 llama3.1 서빙  
  - 현재 터미널에서 nvidia-smi 를 수행해보면 BGE-M3 가 꽤 크게 gpu 메모리를 잡고 있습니다.
  - BGE-M3 모델도 내리고, faiss index 도 저장해주세요.

---

In [ ]:
faiss.write_index(index, "law_index_faiss")

---
### kernel 재시작으로 모두 초기화  
  - jupyter lab 메뉴 > kernel > Restart Kernel
  - kernel 자체가 restart 됐기에, 위에 선언했던 import 내용과 정의한 함수도 모두 초기화 됐습니다.
  - vllm 으로 llama3.1 서빙

---

In [ ]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

# model="Llama-3.2-3B-Instruct",
# model="gemma-3",

def vllm_generate(system_prompt, user_prompt):
    completion = client.chat.completions.create(
        model="Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.95,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [ ]:
from transformers import AutoTokenizer, AutoModel
import os, faiss
from FlagEmbedding import FlagModel
import numpy as np

model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

law_index = faiss.read_index("law_index_faiss")

In [ ]:
queries = [
    "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?",
]

rag_docs = ''
query_embeddings = model.encode_queries(queries)
dists, ids = law_index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

print(rag_docs)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


query:	지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?


후보문장_0:	{
    "판례일련번호": 93952,
    "사건명": "감봉처분취소",
    "사건번호": "78누164",
    "선고일자": "1978-08-22",
    "법원명": "대법원",
    "법원종류코드": null,
    "사건종류명": "일반행정",
    "사건종류코드": 400107.0,
    "판결유형": "판결",
    "선고": "선고",
    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",
    "판시사항": "【판시사항】\n\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",
    "판결요지": "【판결요지】\n\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",
    "참조조문": "【참조조문】\n\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",
    "참조판례": null,
    "판례내용": "【전문】\n\n【원고, 상고인】 원고\n\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\n\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\n\n【주문】\n\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\n\n【이유】\n\n원고의 상고이유를 판단한다.\n\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드

---
## 이때의 오류는 무엇을 의미 하나요?  
  - plain text  vs  embedding vector

---

In [ ]:
directory_path = 'data_test/'
data_list = []
count = 0

# 디렉토리 내 모든 JSON 파일을 읽어들입니다.
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):  # JSON 파일만 읽어들입니다.
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            # JSON 데이터를 텍스트 리스트 형태로 변환합니다.
            text_json = json.dumps(data, indent=4, ensure_ascii=False)
            data_list.append(text_json)
            if count % 10 ==0:
                print(". ", end="")
            count += 1
    else:
        continue

len(data_list)

. . . . . . . . . . . . . . . . 

151

In [ ]:
def rag_serch(query):
    queries = [ query ]

    rag_docs = ''
    query_embeddings = model.encode_queries(queries)
    dists, ids = law_index.search(query_embeddings.astype(np.float32), k=3)

    count = 0
    for i in range(3):
        rag_docs += f"후보문장_{count}:\t{data_list[ids[0][i]]}\n"
    count +=1

    return rag_docs

In [ ]:
rag_serch("지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?")

'후보문장_0:\t{\n    "판례일련번호": 93952,\n    "사건명": "감봉처분취소",\n    "사건번호": "78누164",\n    "선고일자": "1978-08-22",\n    "법원명": "대법원",\n    "법원종류코드": null,\n    "사건종류명": "일반행정",\n    "사건종류코드": 400107.0,\n    "판결유형": "판결",\n    "선고": "선고",\n    "판례상세링크": "/DRF/lawService.do?OC=pwshoot&target=prec&ID=93952&type=HTML&mobileYn=",\n    "판시사항": "【판시사항】\\n\\n지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 당연히 감독의무를 위반하였다고 볼 수 있는지 여부",\n    "판결요지": "【판결요지】\\n\\n지방동무원의 징계에 관한 어느 법규에도 부하직원에게 징계사유가 있다 하여 감독자가 감독의무를 위반한 것으로 본다는 규정이 없고 또 서울시의 지방공무원 징계의 양정에 관한 규칙의 취지도 감독자에게 지방공무원법상의 징계사유가 있음을 전체로 규정된 것이라 할 것이다",\n    "참조조문": "【참조조문】\\n\\n서울특별시지방공무원징계의양정에관한규칙 제2조 제1항, 제2조 제3항",\n    "참조판례": null,\n    "판례내용": "【전문】\\n\\n【원고, 상고인】 원고\\n\\n【피고, 피상고인】 서울특별시장 소송대리인 변호사 장영복\\n\\n【원 판 결】 서울고등법원 1978.3.28. 선고 77구225 판결\\n\\n【주문】\\n\\n원판결을 파기하고 사건을 서울고등법원으로 환송한다.\\n\\n【이유】\\n\\n원고의 상고이유를 판단한다.\\n\\n원심판결에 의하면 원심은 원고가 서울 ○○○구청 위생과 환경계장으로 재직하던 시기인 1975.7월부터 동 12월 사이에 원고의 부하직원이던 소외인이 관내 목욕탕 또는 이발관 등으로부터 단속을 빙자하여 금품을 받고 또는 무허가 드라이영업행위를 적발하

In [ ]:
question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"
#question = "근무태만의 사유로 징계처분 받은것은 적합한가?"

In [ ]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 특히 후보문장 내용 중 "판결요지" 내용들을 바탕으로 질문에 대한 답변을 정확하게 만들어 제공해주세요.
4) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
5) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  question + "\n후보문장: " + rag_serch(question)
vllm_generate(system_prompt, user_prompt)

질문에 대한 답변 후보 문장 중 가장 적합한 후보 문장은 **후보문장_0** 입니다.

**답변:**

지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는 것인가에 대한 질문에 대해, 대법원 102456호 판결(1986.11.25.)은 감독자가 부하직원의 비위사실을 인지하고도 감독의무를 위반한 경우, 그 감독자 또한 징계될 수 있다고 판단했습니다. 

특히, 판결은 금품을 수수한 공무원에 대한 파면 징계가 정당하다는 점을 인정하면서도, 감독자가 부하직원의 위법행위에 대한 감독책임을 지기 위해서는 해당 부하직원이 구체적으로 직무상 태만이나 고의가 있었음을 입증해야 한다고 강조했습니다. 즉, 감독자가 부하직원의 비위사실을 인지하고도 적절한 조치를 취하지 않았다는 점이 입증되어야 감독자 역시 감독의무를 위반한 것으로 인정될 수 있다는 것입니다. 

따라서, 지방동무원의 징계에 관한 법규상 부하직원에게 징계사유가 있더라도 감독자가 감독의무를 위반한 것으로 볼 수 없다는 기존의 해석을 수정하여, 감독자의 감독책임을 강화하는 방향으로 판시되었습니다.

inference time: 12.46074 sec 




---
## 생각해볼 topic  
  - json 덩어리 전체가 embedding 됐는데, 이것이 과연 question 을 embedding 한것과 비교하는것이 적합한것인가?
  - 지금처럼 전처리가 잘 돼 있는 데이터라면, 추가로 시도해볼만한 것은 어떤것이 있을까요?
---

In [ ]:
from datetime import datetime
now = datetime.now()
that_time = datetime.strptime('1991-11-22', '%Y-%m-%d')
time_gap = (now - that_time).days // 30

print('두 날짜의 차')
print(time_gap, '주')

두 날짜의 차
406 주


In [ ]:
import json
from datetime import datetime
# Tool 의 동작 내용을 컴퓨터 언어로 정의 (우리가 이미 잘 알고 있는 python code)

def get_calc_month(date):
    print("판례의 날짜 ({})".format(date))
    if date is None:
      return json.dumps({"date":"정보 부족"})

    now = datetime.now()
    that_time = datetime.strptime(date, '%Y-%m-%d')
    time_gap = (now - that_time).days // 30

    send_info = {
        "time_gap": time_gap,
    }
    print(send_info)
    print("--------------------")
    return json.dumps(send_info)

# Tool 의 동작 내용을 포맷에 맞추어 인간의 언어로 정의 (LLM에게 전달할 내용)
tool_choice = {
    "function": {
        "name": "get_calc_month"
    },
    "type": "function"
}

tools = [{
    "type": "function",
    "function": {
        "name": "get_calc_month",
        "description": "Calculate how many months have passed since a given date.",
        "parameters": {
            "type": "object",
            "properties": {
                "time": {
                    "type": "string",
                    "description": "The date the judgment was announced, e.g. 1991-11-22",
                },
            },
            "required": ["time"],
        },
    },
}]

In [ ]:
def vllm_generate_tool(system_prompt, user_prompt):
    chat_history = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

    # model="Llama-3.2-3B-Instruct",
    # model="gemma-3",

    tool_calls_stream  = client.chat.completions.create(
        model="Llama-3.2-3B-Instruct",
        messages=chat_history,
        tools=tools,
        tool_choice=tool_choice,
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.9,
        max_tokens=1024,
        stream=False,
    )

    tool_call = tool_calls_stream.choices[0].message.tool_calls
    function_response = ""

    if tool_calls_stream.choices[0].message.tool_calls[0].function != None:

        tool_call = tool_calls_stream.choices[0].message.tool_calls[0]

        if tool_call.function:
            if tool_call.function.name:
                print(f"tool call name: {tool_call.function.name}")

            if tool_call.function.arguments:
                print(f"tool call name: {tool_call.function.arguments}")

        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # tool 호출
        if function_name == "get_calc_month":
            function_response = get_calc_month( date=function_args.get("time"))

    chat_history.append(
        {
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    second_completion = client.chat.completions.create(
        model="Llama-3.2-3B-Instruct",
        messages=chat_history,
        frequency_penalty=0.09,
        temperature=0.3,
        top_p=0.9,
        max_tokens=1024,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in second_completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")

In [ ]:
question = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

In [ ]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 후보문장 내용 중 "판결요지" 내용들을 바탕으로 질문에 대한 답변을 정확하게 만들어 제공해주세요.
4) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
5) 특히 선택된 후보문장 중 "선고일자" 를 활용해서 현재부터의 날짜 차이를 개월수로 꼭 표현해줘. ex) 선고된 날짜 현재부터 35개월 전
6) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  question + "\n후보문장: " + rag_serch(question)

vllm_generate_tool(system_prompt, user_prompt)